### 1. What is **Streamlit**?

* Streamlit is a **Python framework** that lets you build interactive web apps **without frontend knowledge** (no need to learn HTML, CSS, JS).
* You write **pure Python** (like you do for ML) → Streamlit automatically turns it into a user-friendly interface.

### 2. Why it’s great for data scientists:

✅ No need to learn web development

✅ Easy to show results, models, and visualizations to non-technical people

✅ Interactive: users can upload data, change parameters, see live results

✅ Free & quick deployment with **Streamlit Cloud**

### 3. How it helps in ML/AI projects:

* You can **wrap your ML model in an app** → others can test it in their browser.
* Example: Instead of showing your CNN in Jupyter Notebook, you give a **web app** where people upload an image and see the prediction.

---



##**🔹 Step 1: Install Streamlit**

In [12]:
pip install streamlit

check if it is correctly installed

In [13]:
!streamlit --version

Streamlit, version 1.49.1


---

##**🔹 Step 2: Basic Streamlit app structure**



In [14]:
# This creates app.py file in your Colab session.
%%writefile app.py

import streamlit as st # importing the streamlit library

# App Title
st.title("My First Streamlit App")

# Displaying simple text
st.write("Hello, this is a test app!")

Overwriting app.py


### run app

In [15]:
!streamlit run app.py &>/dev/null &
#&> → Sends both standard output (logs, messages) And standard error (errors, warnings) to the same place.
#/dev/null → all logs, errors, and messages from Streamlit are hidden.
#appending & at the end of a command = run it in the background.

**Since Streamlit apps run locally inside the notebook and aren’t directly accessible from the internet, we’ll use *pyngrok*. Pyngrok helps us create a secure tunnel between our local app and the internet, giving us a public URL to access the app easily.**


In [16]:
!pip install pyngrok

from pyngrok import ngrok, conf # ngrok → lets us create secure tunnels from Colab to the internet.
                                # conf → is a helper for configuration (like auth token, regions, etc.).

# Set your authtoken
!ngrok config add-authtoken 32mZTZc3h0zhX7v4AHJZTP6PRRP_57q7QafjsjZTMxDinFoqR # saves your personal ngrok auth token.
                                                                              # This token connects Colab to your ngrok account → required for free plan usage.

# Run Streamlit in background
!streamlit run app.py &>/dev/null & # Runs your Streamlit app (app.py) quietly in the background.
                                    #&> → Sends both standard output (logs, messages) And standard error (errors, warnings) to the same place.
                                    #/dev/null → all logs, errors, and messages from Streamlit are hidden.
                                    #appending & at the end of a command = run it in the background.

# Start tunnel
public_url = ngrok.connect(8501) # Now that your app is running on port 8501, ngrok opens a secure tunnel. Exposes port 8501 (where Streamlit is running) to the internet.
print("Your public URL:", public_url)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Your public URL: NgrokTunnel: "https://36c860025b3f.ngrok-free.app" -> "http://localhost:8501"


### 🔹 Step 3: Load your trained CNN model

In [17]:
from google.colab import files
uploaded = files.upload() # choose cnn_image_classifier.keras from your computer which is our pretrained CIFAR-10 image classification file

from tensorflow.keras.models import load_model
import numpy as np
from PIL import Image

# Load model
model = load_model("cnn_image_classifier.keras")

# CIFAR-10 classes
class_names = ["airplane", "automobile", "bird", "cat", "deer",
               "dog", "frog", "horse", "ship", "truck"]

Saving cnn_image_classifier.keras to cnn_image_classifier (2).keras


### 🔹 Step 4: Add Image Upload Feature

In [18]:
%%writefile app.py
import streamlit as st
import numpy as np
from tensorflow.keras.models import load_model
from PIL import Image

# Load your trained CNN model
model = load_model("cnn_image_classifier.keras")

# Class names (for CIFAR-10 example)
class_names = ["airplane", "automobile", "bird", "cat", "deer",
               "dog", "frog", "horse", "ship", "truck"]

# Streamlit App UI
# Adds a title and a small description at the top of the app.
st.title("🖼️ CNN Image Classifier")
st.write("Upload an image and let the model predict the class.")

# File uploader
uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"]) # Restricts uploads to only jpg, jpeg, png image files.
# If the user uploads a file, uploaded_file will contain it; otherwise, it’s None.

if uploaded_file is not None:
    # Open original image
    image = Image.open(uploaded_file)
    st.image(image, caption="Uploaded Image (Original)", use_container_width=True) # Displays the uploaded image in the app with a caption.


    # Preprocess (resize only for model input)
    img_resized = image.resize((32, 32)) # CIFAR-10 uses 32x32 so we will resize the uploaded image. Opens image using Pillow (PIL) and resizes it to 32x32 (since CIFAR-10 expects that size).
    img_array = np.array(img_resized) / 255.0 # Divides by 255.0 to normalize pixel values (CNN models expect inputs between 0 and 1).
    img_array = np.expand_dims(img_array, axis=0) # Adds an extra batch dimension (axis=0) → shape becomes (1, 32, 32, 3) instead of (32, 32, 3).

    # Predict
    predictions = model.predict(img_array) # model.predict(img_array) gives probabilities for all classes.
    predicted_class = class_names[np.argmax(predictions)] # np.argmax(predictions) picks the class with the highest probability.

    st.success(f"Prediction: {predicted_class}")

Overwriting app.py


In [21]:
!streamlit run app.py &>/dev/null &
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print("Streamlit URL:", public_url)


Streamlit URL: NgrokTunnel: "https://ae99b1073417.ngrok-free.app" -> "http://localhost:8501"
